In [ ]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [ ]:

file_path = '목적별 국적별 입국_250708124220.csv'
df = pd.read_csv(file_path, header=0)
df = df.drop(index=0).reset_index(drop=True)

cols = df.columns.tolist()
value_cols = [col for col in cols if ('년' in col and '월' in col and 'Unnamed' not in col)]

long_df = pd.melt(df, id_vars=['국적', '목적'], value_vars=value_cols,
                  var_name='연월', value_name='입국자수')
long_df['연월'] = long_df['연월'].str.replace('년', '-').str.replace('월', '')
long_df['입국자수'] = long_df['입국자수'].astype(str).str.replace(',', '')
long_df['입국자수'] = pd.to_numeric(long_df['입국자수'], errors='coerce')
long_df = long_df[~long_df['목적'].str.strip().isin(['소 계', '소계', '계', '총계'])]
long_df['연월'] = pd.to_datetime(long_df['연월'], format='%Y-%m')
long_df = long_df.dropna()


In [ ]:

lstm_df = long_df.groupby(['연월']).agg({'입국자수': 'sum'}).reset_index().sort_values('연월')

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(lstm_df[['입국자수']])

def create_sequence(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12
X, y = create_sequence(scaled_values, seq_length)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
print(f'LSTM RMSE: {rmse:.2f}')
